In [57]:
import torch
from torch import nn
from utils.fashion_mnist import *

In [58]:
# 定义模型
net = nn.Sequential(nn.Flatten(),
                    nn.Linear(28 * 28, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10),
                   ).cuda()

def init_weights(m):
    if type(m) == nn.Linear:
        # 添加下划线的normal_是一个原地操作，直接修改该变量，而不是新建
        nn.init.normal_(m.weight, std=0.5)
        
net.apply(init_weights)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)

In [59]:
batch_size, lr, num_epochs = 32, 0.1, 50
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(net.parameters(), lr=lr)

In [60]:
train_iter, test_iter = load_data_fashion_mnist(batch_size)

for i in range(num_epochs):
    train_info = [0.0, 0.0, 0]
    net.train()
    for X, y in train_iter:
        X, y = X.cuda(), y.cuda()
        y_hat = net(X)
        l = loss(y_hat, y)
        trainer.zero_grad()
        l.mean().backward()
        trainer.step()
        
        train_info = [a + float(b) for a, b in zip(train_info, [float(l.sum()), accuracy(y_hat, y), y.numel()])]
    
    print(f'loss: {train_info[0] / train_info[2]}, acc: {train_info[1] / train_info[2]}')

loss: 0.030255204596494636, acc: 0.7786333333333333
loss: 0.015412324227268497, acc: 0.8284833333333333
loss: 0.013789862857945263, acc: 0.84395
loss: 0.012836863717933496, acc: 0.8534
loss: 0.012096091123794516, acc: 0.8599
loss: 0.011585892386734485, acc: 0.8663166666666666
loss: 0.011129693173492949, acc: 0.87055
loss: 0.01077446008225282, acc: 0.8739
loss: 0.010421471600731215, acc: 0.87815
loss: 0.010113734938173245, acc: 0.8820666666666667
loss: 0.009862567800780138, acc: 0.8834833333333333
loss: 0.009627468968865772, acc: 0.8867333333333334
loss: 0.009401046283977726, acc: 0.8892166666666667
loss: 0.009168797106295824, acc: 0.8926833333333334
loss: 0.008971286585306128, acc: 0.8938833333333334
loss: 0.008807873623166233, acc: 0.8962166666666667
loss: 0.008670178794239957, acc: 0.8983333333333333
loss: 0.008458965981813768, acc: 0.89995
loss: 0.008337706105473141, acc: 0.9012
loss: 0.008187697355511288, acc: 0.9023666666666667
loss: 0.008040723213367164, acc: 0.9052333333333333
l

KeyboardInterrupt: 

In [27]:
predict_ch3(net, test_iter)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [25]:
test_info = [0.0, 0]

for X, y in test_iter:
    y_hat = net(X)
    test_info = [a + float(b) for a, b in zip(test_info, [accuracy(y_hat, y), y.numel()])]
    
print(f'test acc: {test_info[0] / test_info[1]}')

test acc: 0.8617
